In [1]:
library(circlize)


circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))




In [2]:
# 1. Prepare the Matrix
prepare_matrix <- function(file_path) {
  mat <- as.matrix(read.table(file_path, sep = ";"))
  
  # Initialize empty matrices for uncertain values
  incognites <- matrix(0, nrow(mat), ncol(mat))
  incognites_pos <- incognites_neg <- incognites_zeros <- incognites
  
  # Mark uncertain links
  incognites[mat %in% c(0.1, 0.5, 0.9)] <- 1
  incognites_pos[mat == 0.9] <- 1
  incognites_neg[mat == 0.1] <- 1
  incognites_zeros[mat == 0.5] <- 1
  
  # Set uncertain values to 0 in main matrix to avoid double-counting
  mat[mat %in% c(0.1, 0.5, 0.9)] <- 0
  
  list(mat = mat, incognites = incognites, 
       incognites_pos = incognites_pos, 
       incognites_neg = incognites_neg, 
       incognites_zeros = incognites_zeros)
}

mats <- prepare_matrix("../../data/matrix_decoded.csv")

In [3]:
# 2. Prepare the data

prepare_data <- function(mat) {
    items <- 1:nrow(mat)
    v1 <- c(0, cumsum(apply(mat, 1, function(x) sum(abs(x)))))
    df <- data.frame(
        factor = rep(items, each = 2), 
        x = rep(v1, each = 2)[-c(1, length(v1) * 2)]
        )
    data_start_end <- data.frame(
        factor = items,
        start = v1[1:length(v1)-1],
        end = v1[2:length(v1)]
    )
    return(list(df = df, data_start_end = data_start_end))
} 

data <- prepare_data(mats$mat)


In [9]:
draw_links <- function(mats, data_start_end, plot_reciprocals, plot_nonreciprocals, plot_unknowns) {
    red = rgb(255/255, 51/255, 0/255, 0.8)
    blue = rgb(51/255, 51/255, 255/255, 0.8)
    pink = rgb(255/255, 153/255, 0/255, 0.8)
    green = rgb(0/255, 153/255, 51/255, 0.8)

    for (i in 1:nrow(mats$mat)) {
        i0 = data_start_end$start[i] # start of the arc at the source


        for (j in i:ncol(mats$mat)) {
            if (i == j) next # skip the diagonal
            # print the source and target
            j0 = data_start_end$start[j] # start of the arc at the target
            val_ij = mats$mat[i,j]
            val_ji = mats$mat[j,i]
            width_ij = abs(val_ij)
            width_ji = abs(val_ji)


            # color is red for positive values, blue for negative values, black for 0
            col_ij = ifelse(val_ij > 0, red, ifelse(val_ij < 0, blue, "black"))
            col_ji = ifelse(val_ji > 0, red, ifelse(val_ji < 0, blue, "black"))

            if (col_ij == col_ji & col_ij == red) {
                # Reciprocal positive
                col_ij = col_ji = pink
                if (plot_reciprocals) {
                    circos.link(i, c(i0, i0+width_ij), j, c(j0, j0+width_ji), col = col_ij, lwd = 1, lty = 1, border="black")
                }
            } else if (col_ij == col_ji & col_ij == blue) {
                # Reciprocal negative
                col_ij = col_ji = green
                if (plot_reciprocals) {
                    circos.link(i, c(i0, i0+width_ij), j, c(j0, j0+width_ji), col = col_ij, lwd = 1, lty = 1, border="black")
                }
            } else {
                # Non-reciprocal
                if (width_ij > 0 & width_ji > 0) {
                    if (plot_nonreciprocals) {
                        circos.link(i, c(i0, i0+width_ij), j, c(j0+width_ji, j0+width_ji), col = col_ij)
                        circos.link(i, c(i0+width_ij, i0+width_ij), j, c(j0, j0+width_ji), col = col_ji)
                        circos.link(i, c(i0, i0+width_ij), j, c(j0, j0+width_ji), col = rgb(0,0,0,0), lwd = 1, lty = 1, border="black" )
                    }

                } else if (width_ij > 0 | width_ji > 0) {

                    col_ij = ifelse(val_ij > 0, pink, col_ij)
                    col_ij = ifelse(val_ij < 0, green, col_ij)
                    col_ji = ifelse(val_ji > 0, pink, col_ji)
                    col_ji = ifelse(val_ji < 0, green, col_ji)
                    # edge in one direction
                    if (plot_reciprocals) {
                        if (width_ij > 0) circos.link(i, c(i0, i0+width_ij), j, c(j0+width_ji, j0+width_ji), col = col_ij, lwd = 1, lty = 1, border="black" )
                        if (width_ji > 0) circos.link(i, c(i0+width_ij, i0+width_ij), j, c(j0, j0+width_ji), col = col_ji, lwd = 1, lty = 1, border="black" )
                    }   
                } else if (width_ij == 0 & mats$incognites[i,j] == 1) {
                    # incognite
                    if (plot_unknowns) {
                        col = ifelse(mats$incognites_pos[i,j] == 1, red, ifelse(mats$incognites_neg[i,j] == 1, blue, "black"))
                        circos.link(i, c(i0, i0), j, c(j0, j0), col = col, lwd = 2,lty = 1, directional = 1)
                    }
                } else if (width_ji == 0 & mats$incognites[j,i] == 1) {
                    # incognite
                    if (plot_unknowns) {
                        col = ifelse(mats$incognites_pos[j,i] == 1, red, ifelse(mats$incognites_neg[j,i] == 1, blue, "black"))
                        circos.link(j, c(j0, j0), i, c(i0, i0), col = col, lwd = 2,lty = 1, directional = 1)
                    }
                }
            }
            
            # update the start of the arc at the target
            data_start_end$start[j] = j0 + width_ji
            i0 = i0 + width_ij
        }
        data_start_end$start[i] = i0
    }
}

In [10]:
initialize_circos <- function(mats, df, data_start_end) {
    total_gap <- 120
    gap_degree <- total_gap / nrow(mats$mat)
    circos.par(gap.degree = gap_degree, "canvas.xlim" = c(-1, 1), "canvas.ylim" = c(-1, 1))
    circos.par(cell.padding = c(0.02, 0, 0.02, 0))
    circos.initialize(factors = df$factor, x = df$x)
    # Generate color palette
    atmos_col = rgb(153/255, 0/255, 153/255, 0.6)
    land_ocean_col = rgb(255/255, 153/255, 0, 0.6)
    phy_chem_col = rgb(102/255, 153/255, 255/255, 0.6)
    bio_col = rgb(230/255, 230/255, 0/255, 0.6)
    human_col = rgb(254/255, 0/255, 128/255, 0.6)
    cols <- c(rep(atmos_col, 5),
                rep(land_ocean_col, 4),
                rep(phy_chem_col, 6),
                rep(bio_col, 3),
                rep(human_col, 5))
    
    circos.trackPlotRegion(factors = df$factor, ylim = c(0, 1), bg.col = cols, 
                            bg.border = NA, track.height = 0.05)
    h.ratio = 0.7
}

In [11]:
add_names <- function(names, data_start_end) {
  for (i in 1:nrow(data_start_end)) {
        circos.text(x = (data_start_end$start[i] + data_start_end$end[i]) / 2, 
            y = 3, 
            sector.index = data_start_end$factor[i],
            labels = names[i], 
            facing = "bending.inside", 
            niceFacing = TRUE, 
            adj = c(0.5, 0.5), 
            cex = 0.6
            )
  }
}

In [15]:


file_name <- function(folder, plot_reciprocals, plot_nonreciprocals, plot_unknowns, extension) {
    # if all are true, the file name is "all" + .extension
    if (plot_reciprocals & plot_nonreciprocals & plot_unknowns) {
        return(paste(folder, "all", sep = ""))
    }
    # if only reciprocals are plotted, the file name is "reciprocals" + .extension
    if (plot_reciprocals & !plot_nonreciprocals & !plot_unknowns) {
        return(paste(folder, "reciprocals", sep = ""))
    }
    # if only non-reciprocals are plotted, the file name is "nonreciprocals" + .extension
    if (!plot_reciprocals & plot_nonreciprocals & !plot_unknowns) {
        return(paste(folder, "nonreciprocals", sep = ""))
    }
    # if only unknowns are plotted, the file name is "unknowns" + .extension
    if (!plot_reciprocals & !plot_nonreciprocals & plot_unknowns) {
        return(paste(folder, "unknowns", sep = ""))
    }
}


In [27]:
circos.clear()
plot_reciprocals = TRUE
plot_nonreciprocals = TRUE
plot_unknowns = TRUE

circos_plot <- function(mats, df, data_start_end, plot_reciprocals, plot_nonreciprocals, plot_unknowns) {
    fn = paste(file_name("../../figures/", plot_reciprocals, plot_nonreciprocals, plot_unknowns, "pdf"), ".pdf", sep="")
    pdf(fn)
    initialize_circos(mats, df, data_start_end)
    add_names(read.csv("../../data/Variables_names.csv")$Acr1, data_start_end)
    draw_links(mats, data_start_end, plot_reciprocals, plot_nonreciprocals, plot_unknowns)
    dev.off()

    fn = paste(file_name("../../figures/", plot_reciprocals, plot_nonreciprocals, plot_unknowns, "png"), ".png", sep="")
    png(fn, width = 1500, height = 1500, res = 300)
    initialize_circos(mats, df, data_start_end)
    add_names(read.csv("../../data/Variables_names.csv")$Acr1, data_start_end)
    draw_links(mats, data_start_end, plot_reciprocals, plot_nonreciprocals, plot_unknowns)
    dev.off()
}

circos_plot(mats, data$df, data$data_start_end, plot_reciprocals, plot_nonreciprocals, plot_unknowns)


Note: 1 point is out of plotting region in sector '1', track '1'.

Note: 1 point is out of plotting region in sector '1', track '1'.

Note: 2 points are out of plotting region in sector '1', track '1'.

Note: 1 point is out of plotting region in sector '2', track '1'.

Note: 1 point is out of plotting region in sector '2', track '1'.

Note: 2 points are out of plotting region in sector '2', track '1'.

Note: 1 point is out of plotting region in sector '3', track '1'.

Note: 1 point is out of plotting region in sector '3', track '1'.

Note: 2 points are out of plotting region in sector '3', track '1'.

Note: 1 point is out of plotting region in sector '4', track '1'.

Note: 1 point is out of plotting region in sector '4', track '1'.

Note: 3 points are out of plotting region in sector '4', track '1'.

Note: 1 point is out of plotting region in sector '5', track '1'.

Note: 1 point is out of plotting region in sector '5', track '1'.

Note: 3 points are out of plotting region in sector '5

png 
  2

In [52]:
# initialize a new plot
# set canvas size
pdf("../../figures/legend.pdf",width = 4, height = 2)
par(mar=c(0,0,0,0))
plot.new()



red = rgb(255/255, 51/255, 0/255, 0.8)
blue = rgb(51/255, 51/255, 255/255, 0.8)
pink = rgb(255/255, 153/255, 0/255, 0.8)
green = rgb(0/255, 153/255, 51/255, 0.8)

atmos_col = rgb(153/255, 0/255, 153/255, 0.6)
land_ocean_col = rgb(255/255, 153/255, 0, 0.6)
phy_chem_col = rgb(102/255, 153/255, 255/255, 0.6)
bio_col = rgb(230/255, 230/255, 0/255, 0.6)
human_col = rgb(254/255, 0/255, 128/255, 0.6)


legend(
"topleft",
legend = c("Atmosphere", "Land and Ocean", "Physical and Chemical", "Biological", "Human"),
col = c(atmos_col, land_ocean_col, phy_chem_col, bio_col, human_col),
lty = c(1, 1, 1, 1, 1),
lwd = c(8, 8, 8, 8, 8),
bty = "n",
cex = 1.1,
ncol = 1
)

legend(
"topright",
legend = c("pos->neg", "neg->pos", "pos->pos", "neg->neg", ">0", "<0", "~0"),
col = c(red, blue, pink, green, red, blue, "black"),
lty = c(1, 1, 1, 1, 1, 1, 1),
lwd = c(8, 8, 8, 8, 1, 1, 1),
bty = "n",
cex = 1.1,
ncol = 1
)

dev.off()



png 
  2